In [1]:
from datetime import date, datetime, timedelta
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [24]:
# multivariate multi-step encoder-decoder lstm for the power usage dataset
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed


# split a univariate dataset into train/test sets
def split_dataset(data):
    splitRatio = int(2*len(data)/3)
    # split into standard weeks
    train, test = data[:splitRatio], data[splitRatio:]
    # restructure into windows of weekly data
    train = array(split(train, len(train)/24))
    test = array(split(test, len(test)/24))
    return train, test


# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        #mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        #rmse = sqrt(mse)
        # store
        #scores.append(rmse)
        #calculate mae
        mae = mean_absolute_error(actual[:, i], predicted[:, i])
        scores.append(mae)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores


# summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))
    
    
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=24):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)


# train the model
def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 1, 30, 5
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model


# make a forecast
def forecast(model, history, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, :]
    # reshape into [1, n_input, n]
    input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat


# evaluate a single model
def evaluate_model(train, test, n_input):
    # fit model
    model = build_model(train, n_input)
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
        yhat_sequence = forecast(model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
    # evaluate predictions days for each week
    predictions = array(predictions)
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores, predictions

In [20]:
# load cleaned and prepared data in EDA
numDf = pd.read_csv("zoneFeatures.csv", index_col="time_bin", parse_dates=["time_bin"])
catDf = pd.read_csv("catFeatures.csv", index_col="time_bin", parse_dates=["time_bin"])

In [21]:
df = numDf[["132"]].join(catDf)

In [73]:
#df["lag"] = df["132"].shift(168)

In [74]:
#df = df.fillna(0)

In [ ]:
n_input = 3
train, test = split_dataset(df.values)
score, scores, pred = evaluate_model(train, test, n_input)

Epoch 1/30
3499/3499 [==============================] - 158s 45ms/step - loss: 231.0331
Epoch 2/30
3499/3499 [==============================] - 143s 41ms/step - loss: 89.0795
Epoch 3/30
3499/3499 [==============================] - 141s 40ms/step - loss: 81.7230
Epoch 4/30
3499/3499 [==============================] - 142s 41ms/step - loss: 74.6202
Epoch 5/30
3499/3499 [==============================] - 148s 42ms/step - loss: 66.5799
Epoch 6/30
3499/3499 [==============================] - 144s 41ms/step - loss: 61.2029
Epoch 7/30
3499/3499 [==============================] - 149s 43ms/step - loss: 57.5558
Epoch 8/30
3499/3499 [==============================] - 140s 40ms/step - loss: 59.2110
Epoch 9/30
3499/3499 [==============================] - 140s 40ms/step - loss: 54.7097
Epoch 10/30
3499/3499 [==============================] - 137s 39ms/step - loss: 52.8388
Epoch 11/30
3499/3499 [==============================] - 136s 39ms/step - loss: 51.6986
Epoch 12/30
3499/3499 [=================

In [8]:
yPredList = [item for sublist in pred.tolist() for item in sublist]
yPredList1 = [item for sublist in yPredList for item in sublist]

In [9]:
mean_absolute_error(df[-5256:]["132"].values, yPredList1)

55.39672862521771

In [50]:
len(yPredList1)

5256

In [53]:
df[-5256:]["132"]

time_bin
2019-05-27 00:00:00    283.0
2019-05-27 01:00:00    185.0
2019-05-27 02:00:00     42.0
2019-05-27 03:00:00     15.0
2019-05-27 04:00:00     48.0
                       ...  
2019-12-31 19:00:00    459.0
2019-12-31 20:00:00    472.0
2019-12-31 21:00:00    369.0
2019-12-31 22:00:00    324.0
2019-12-31 23:00:00    296.0
Name: 132, Length: 5256, dtype: float64

In [10]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [22]:
df.year -= df.year.min()  # equivalent to df = df - df.min()
df.year /= df.year.max()  # equivalent to df = df / df.max()

In [23]:
df

,132,year,holiday,temp,month_2,month_3,month_4,month_5,month_6,month_7,...,hour_20,hour_21,hour_22,hour_23,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
time_bin,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,263.0,0.0,1.0,6.87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2017-01-01 01:00:00,124.0,0.0,1.0,7.02,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2017-01-01 02:00:00,53.0,0.0,1.0,6.73,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2017-01-01 03:00:00,36.0,0.0,1.0,6.80,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2017-01-01 04:00:00,20.0,0.0,1.0,6.38,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,459.0,1.0,0.0,6.32,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2019-12-31 20:00:00,472.0,1.0,0.0,6.32,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2019-12-31 21:00:00,369.0,1.0,0.0,6.32,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
